# Sistema Interconectado Nacional de Energía Eléctrica de Colombia (SIN)
EL Sistema Interconectado Nacional es el sistema compuesto por los siguientes elementos conectados entre sí: las plantas y equipos de generación, la red de interconexión, las redes regionales e interregionales de transmisión, las redes de distribución, y las cargas eléctricas de los usuarios. [XM](https://www.xm.com.co/corporativo/Paginas/Herramientas/glosario.aspx)

En el presente caso de estudio se pretende mostrar las principales variables que interactuan en el SIN e inciden en los mercado de Energía Eléctrica y Gas Natural




In [2]:
from bs4 import BeautifulSoup as bfs4
import pandas as pd 
import numpy as np 
import matplotlib as plt
import requests as rq
from datetime import date
from datetime import datetime
import requests

In [3]:
from collections import Counter
from itertools import chain
import time
from copy import deepcopy
from concurrent import futures

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options # for headless later

In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Downloading source file from [XM_PortalBI](http://portalbissrs.xm.com.co/oferta/Histricos/Generacion_(kWh)_TRIM1_2020.xlsx)

In [59]:
now = datetime.now()
year = now.strftime('%Y')
month = now.strftime('%m')
day = now.strftime('%d')
day = int(day)
if (day-3) < 10:
    dayres3 = "0" + str(day - 3) #3 days delay
else:
    dayres3 = str(day - 3) #3 days delay

m = int(month)
if m >= 1 and m <=3:
    trim = "1"
elif m >= 4 and m <=6:
    trim = "2"
elif m >= 7 and m <=9:
    trim = "3"
else:
    trim = "4"

name = '{0}_{1}_{2}_Generacion_(kWh)_TRIM{3}_{0}.xlsx'.format(year,month,dayres3,trim)
linkgen = "http://portalbissrs.xm.com.co/oferta/Histricos/Generacion_(kWh)_TRIM_{0}_{1}.xlsx".format(trim,year)

resp = requests.get(linkgen)
output = open(name, 'wb')
output.write(resp.content)
output.close()
print("Archivo descargado:",name)


722

Archivo descargado: 2020_10_22_Generacion_(kWh)_TRIM4_2020.xlsx


In [1]:
xlsx_ol='http://portalbissrs.xm.com.co/oferta/_layouts/15/WopiFrame.aspx?sourcedoc={E3A19B33-E373-4CDC-ABA8-BE815407EDE2}&file=Generacion_(kWh)_TRIM4_2020.xlsx&action=default'

In [8]:
r = requests.get(xlsx_ol)

In [11]:
r.text

'\r\n<!DOCTYPE html>\r\n<html>\r\n<head>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n\t<meta name=\'viewport\' content=\'width=device-width, initial-scale=1\' />\r\n\t<meta name=\'robots\' content=\'noindex\' />\r\n\t<link rel="shortcut icon" href="http://officeonlinexm.xm.com.co/x/_layouts/resources/FavIcon_Excel.ico" />\r\n\t<title>Generacion_(kWh)_TRIM4_2020.xlsx</title>\r\n\t\r\n\t<script type="text/javascript">\r\n\t   var WOPIPerf_UserClick = null;\r\n\t   if (window.sessionStorage)\r\n\t   {\r\n\t\t   WOPIPerf_UserClick = window.sessionStorage.getItem("WOPIPerf_UserClickTime");\r\n\t\t   window.sessionStorage.removeItem("WOPIPerf_UserClickTime");\r\n\t   }\r\n\t</script>\r\n\t\r\n\t<script type="text/javascript">\r\nfunction ULS6zp(){var o=new Object;o.ULSTeamName="Microsoft SharePoint Foundation";o.ULSFileName="WOPIFrame.aspx";return o;}\r\n\t\tfunction getWopiIFrameElement()\r\n\t\t{ULS6zp:;\r\n\t\t\treturn document.querySelector("iframe[name=WebApplication

In [ ]:
archivo_excel = pd.read_table(name)
#archivo_excel = pd.read_excel(open(name,'rb'))
#xl = pd.ExcelFile(name)
#print(archivo_excel.columns)
print(archivo_excel)
#values = archivo_excel['Identificador'].values
#print(values)

In [ ]:
os.rename(name, name[0:-5]+'.csv')
csv = pd.read_html(name[0:-5]+'.csv')#, error_bad_lines=False)

In [ ]:
csv

In [56]:
column_names = ['Fecha','Recurso','Tipo Generación','Combustible','Código Agente','Tipo Despacho','Es Menor','Clasificación','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','Version']#,'34','35','36']

# Selenium cells

In [5]:
driver = webdriver.Chrome()
driver.get('http://portalbissrs.xm.com.co/oferta/_layouts/15/WopiFrame.aspx?sourcedoc={E3A19B33-E373-4CDC-ABA8-BE815407EDE2}&file=Generacion_(kWh)_TRIM4_2020.xlsx&action=default')

driver.switch_to.frame(0)

In [8]:

xpaths = ['//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_0"]',
          '//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_1"]',
          '//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_2"]',
          '//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_3"]',
          '//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_4"]']

In [11]:

e = driver.find_element_by_xpath(xpaths[0])
# Load next part of DOM to avoid NoSuchElementException: Message: no such element: Unable to locate element
driver.execute_script("arguments[0].scrollIntoView();", e)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="m_excelWebRenderer_ewaCtl_sheetContentDiv_Flow_0"]"}
  (Session info: chrome=86.0.4240.111)


In [50]:
e_header = driver.find_element_by_xpath('//*[@id="gridRows"]/div/div/div') # do not use find_elements_by_xpath, only want 1st xpath

# 'RAMA\nAGENCIA7\nAGENCIA\nNOMBRE_AGENCIA\nNUM_PROGRAMA\nNOMBRE_PROGRAMA\nFONDO\nCODIGO_CONCEPTO\nNUM_CUENTA\nDESCRIPCION\nJOINT_RESOLUTION_CATEGORY\nAF_2020_APR\nAF_2021_FOMB Baseline\nAF_2021_Propuesto Gob'
column_names = e_header.text.split('\n') 

column_names

['Generación (kWh) 2020']

In [57]:
def parse_block(row_block):
    """Parse webelement to iterables grouped into iterable of 14 for EDA purposes"""
    iterable = row_block.t
    ext.split('\n')
    return zip(*[iter(iterable)]*7)

In [58]:
pd.DataFrame(parse_block(e),columns=column_names)  # https://docs.python.org/3/library/itertools.html#itertools-recipespd.DataFrame(parse_block(e),columns=column_names)  # https://docs.python.org/3/library/itertools.html#itertools-recipes

,Fecha,Recurso,Tipo Generación,Combustible,Código Agente,Tipo Despacho,Es Menor,Clasificación,0,1,...,15,16,17,18,19,20,21,22,23,Version
0,Generación (kWh) 2020,Fecha,Recurso,Tipo Generación,Combustible,Código Agente,Tipo Despacho,Es Menor,Clasificación,0,...,14,15,16,17,18,19,20,21,2020-10-01,AGUA FRESCA
1,HIDRAULICA,AGUA,ADCG,NO DESPACHADO CENTRALMENTE,SI,NORMAL,"7.293,60","7.294,80","7.296,00","7.296,00",...,"7.290,00","7.287,60","7.290,00","7.291,20","7.293,60",2020-10-01,ALBAN,HIDRAULICA,AGUA,EPSG
2,DESPACHADO CENTRALMENTE,NO,NORMAL,"239.215,00","237.701,00","279.342,00","269.513,00","268.592,00","269.285,00","269.006,00",...,"415.760,00","415.300,00",2020-10-01,ALEJANDRÍA,HIDRAULICA,AGUA,GALG,NO DESPACHADO CENTRALMENTE,SI,NORMAL
3,"12.232,00","12.287,00","13.596,00","14.993,00","14.938,00","14.960,00","14.696,00","14.432,00","14.058,00","13.519,00",...,ALTO TULUA,HIDRAULICA,AGUA,EPSG,NO DESPACHADO CENTRALMENTE,SI,NORMAL,"1.555,20","1.659,60","1.846,80"
4,"1.846,80","1.846,80","1.843,20","1.846,80","1.846,80","1.850,40","1.846,80","1.846,80","1.627,20","1.472,40",...,EPSG,NO DESPACHADO CENTRALMENTE,SI,NORMAL,"2.221,20","2.347,20","2.347,20","2.383,20","2.437,20","2.440,80"
5,"2.444,40","2.444,40","2.422,80","2.149,20","2.145,60","2.149,20","2.149,20","1.947,60","1.944,00","1.843,20",...,NORMAL,"651,26","651,22","651,09","650,23","648,74","413,97","24,79","636,68","643,20"
6,"642,96","641,38","648,52","647,22","647,99","640,43","647,67","647,33","646,26","646,31",...,"56,32","11,44","61,51","13,82","0,79",2020-10-01,AMOYA LA ESPERANZA,HIDRAULICA,AGUA,ISGG
7,DESPACHADO CENTRALMENTE,NO,FILO DE AGUA,"63.456,92","64.695,85","64.754,75","64.731,48","63.939,31","62.705,74","61.992,55",...,"50.813,68","50.322,14",2020-10-01,ASNAZU,HIDRAULICA,AGUA,GNCG,NO DESPACHADO CENTRALMENTE,SI,NORMAL
8,"484,72","484,61","484,63","484,61","484,85","484,76","484,72","484,73","484,69","484,54",...,AURES BAJO,HIDRAULICA,AGUA,ABAG,NO DESPACHADO CENTRALMENTE,SI,NORMAL,"16.123,90","16.111,90","16.101,40"
9,"16.077,70","16.093,20","16.103,00","16.100,90","16.088,40","16.065,40","16.072,40","16.075,30","16.075,20","16.071,60",...,EPMG,NO DESPACHADO CENTRALMENTE,NO,AUTOGENERADOR,"6.174,67","6.189,33","6.211,33","6.138,00","6.182,00","6.288,33"
